
"""
Plan A：Embedding×2 + Conv1D + 2 couches GRU
"""
Mais ce modèle est trop nul donc on lui a abandonné

In [9]:

import os, glob, random, re, shutil, collections, pathlib
import numpy as np, tensorflow as tf, keras
from sklearn.metrics import f1_score


tf.random.set_seed(0)
np.random.seed(0)


In [10]:

# -------- 1. chemin ----------
DATA_DIR = pathlib.Path("./dataset")
TRAIN_DIR, DEV_DIR, TEST_DIR = DATA_DIR/"train", DATA_DIR/"dev", DATA_DIR/"test"
PRED_DIR = DATA_DIR/"pred"; PRED_DIR.mkdir(exist_ok=True)


# if not DEV_DIR.exists() or not any(DEV_DIR.iterdir()):
#     DEV_DIR.mkdir(exist_ok=True)
#     files = sorted(TRAIN_DIR.glob("*.txt")); random.shuffle(files)
#     for f in files[: int(0.1*len(files))]:
#         shutil.move(str(f), DEV_DIR/f.name)


In [11]:

# -------- 2.découpage de phrases  ----------
CH_PUNCTS = "，。、：；？！「」"
def strip_punct(s): return re.sub(f"[{CH_PUNCTS}]", "", s)

def read(folder, with_punct=True):
    s, t = [], []
    for fp in sorted(folder.glob("*.txt")):
        txt = fp.read_text(encoding="utf-8").strip()
        if not txt: continue
        s.append(strip_punct(txt) if with_punct else txt)
        t.append(txt if with_punct else "")
    return s, t

train_src, train_tgt = read(TRAIN_DIR, True)
dev_src,   dev_tgt   = read(DEV_DIR,   True)
test_src,  _         = read(TEST_DIR,  False)


In [12]:

# -------- 3. TextVectorization ----------
tv = keras.layers.TextVectorization(
        standardize=None, split="character",
        output_mode="int", output_sequence_length=256)
tv.adapt(train_src + train_tgt + dev_src + dev_tgt)
VOCAB, PAD = tv.vocabulary_size(), 0


def shift_right(tensor):               
    pad = tf.zeros_like(tensor[:, :1]) 
    return tf.concat([pad, tensor[:, :-1]], axis=1)

def vec(src_list, tgt_list):
    x = tv(np.array(src_list))         
    if tgt_list[0]:                    
        y = tv(np.array(tgt_list))     
        y = shift_right(y)            
        m = tf.cast(y != PAD, "float32")
        return x, y, m
    else:                              # test
        return x, None, None


x_tr,y_tr,m_tr = vec(train_src,train_tgt)
x_dev,y_dev,m_dev = vec(dev_src,dev_tgt)
x_te,_,_ = vec(test_src,[""]*len(test_src))


In [13]:

# -------- 4. Modèle ----------
def build_model(vocab, emb_dim=256, conv_filters=256, gru_dim=256, drop=0.3):   
    inp = keras.Input((None,), dtype="int32")
    x   = keras.layers.Embedding(vocab, emb_dim)(inp)

    x2  = keras.layers.Conv1D(conv_filters, 3, padding="causal", activation="relu")(x)
    x   = keras.layers.Add()([x, x2])
    x   = keras.layers.LayerNormalization()(x); x = keras.layers.Dropout(drop)(x)

    g1  = keras.layers.GRU(gru_dim, return_sequences=True)(x)
    g2  = keras.layers.GRU(gru_dim, return_sequences=True)(g1)
    x   = keras.layers.Add()([g1, g2])                     # residual
    x   = keras.layers.LayerNormalization()(x); x = keras.layers.Dropout(drop)(x)

    out = keras.layers.Dense(vocab, activation="softmax")(x)
    model = keras.Model(inp, out)
    loss  = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(
        optimizer=keras.optimizers.Adam(3e-4),            
        loss=loss,
        metrics=[keras.metrics.SparseCategoricalAccuracy()])
    return model

model = build_model(VOCAB)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │  1,246,720 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, None, 256) │    196,864 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, None, 256) │          0 │ embedding_1[0][0… │
│                     │                   │            │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 256) │        512 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, None, 256) │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ (None, None, 256) │    394,752 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_3 (GRU)         │ (None, None, 256) │    394,752 │ gru_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, None, 256) │          0 │ gru_2[0][0],      │
│                     │                   │            │ gru_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 256) │        512 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 256) │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  1,251,590 │ dropout_3[0][0]   │
│                     │ 4870)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,485,702 (13.30 MB)

 Trainable params: 3,485,702 (13.30 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:

# -------- 5. sample_weight ----------
sw_tr, sw_dev = m_tr.numpy(), m_dev.numpy()


In [15]:

# -------- 6. Train ----------
def cosine_lr(epoch, lr0=3e-4, total=30):
    import math
    return lr0 * (1 + math.cos(epoch/total * math.pi)) / 2  

history = model.fit(
    x_tr, y_tr, sample_weight=sw_tr,
    validation_data=(x_dev, y_dev, sw_dev),
    batch_size=32, epochs=30, verbose=2,
    callbacks=[
        keras.callbacks.LearningRateScheduler(cosine_lr),
        keras.callbacks.EarlyStopping(monitor="val_loss",
                                      patience=5, restore_best_weights=True)
    ])


Epoch 1/30
3/3 - 5s - 2s/step - loss: 8.4519 - sparse_categorical_accuracy: 4.3403e-04 - val_loss: 8.3797 - val_sparse_categorical_accuracy: 0.0063 - learning_rate: 3.0000e-04
Epoch 2/30
3/3 - 4s - 1s/step - loss: 8.3186 - sparse_categorical_accuracy: 0.0032 - val_loss: 8.2236 - val_sparse_categorical_accuracy: 0.0410 - learning_rate: 2.9918e-04
Epoch 3/30
3/3 - 4s - 1s/step - loss: 8.1404 - sparse_categorical_accuracy: 0.0299 - val_loss: 7.9795 - val_sparse_categorical_accuracy: 0.0830 - learning_rate: 2.9591e-04
Epoch 4/30
3/3 - 3s - 1s/step - loss: 7.8779 - sparse_categorical_accuracy: 0.0799 - val_loss: 7.6699 - val_sparse_categorical_accuracy: 0.0884 - learning_rate: 2.8867e-04
Epoch 5/30
3/3 - 3s - 1s/step - loss: 7.5585 - sparse_categorical_accuracy: 0.0921 - val_loss: 7.3869 - val_sparse_categorical_accuracy: 0.0889 - learning_rate: 2.7619e-04
Epoch 6/30
3/3 - 3s - 1s/step - loss: 7.2700 - sparse_categorical_accuracy: 0.0922 - val_loss: 7.1698 - val_sparse_categorical_accuracy:

In [16]:

# -------- 7. Dev macro-F1 ----------
def macro_f1(y_true, y_prob, mask):
    y_pred = y_prob.argmax(-1)
    yt, yp = [], []
    for t,p,m in zip(y_true.numpy(), y_pred, mask.numpy().astype(bool)):
        yt.extend(t[m]); yp.extend(p[m])
    return f1_score(yt, yp, average="macro", zero_division=0)

print("Dev macro-F1:", macro_f1(y_dev, model.predict(x_dev,32,0), m_dev))


Dev macro-F1: 0.0002326937239178463


In [17]:

# # -------- 8. Inference ----------
# chars = tv.get_vocabulary()
# def ids2txt(ids): return "".join(chars[i] for i in ids if i!=PAD)

# for fp, src, logits in zip(sorted(TEST_DIR.glob("*.txt")), test_src,
#                            model.predict(x_te,32,0)):
#     txt = ids2txt(logits.argmax(-1))
#     if not any(p in txt for p in CH_PUNCTS): txt = src   # fallback
#     (PRED_DIR/fp.name).write_text(txt, encoding="utf-8")

# print("Saved at", PRED_DIR)